<a href="https://colab.research.google.com/github/kr-vivekk/quantumML/blob/main/QCNN_USING_PYTORCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane
!pip install quimb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

import os
import random
import cv2
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.utils import to_categorical

In [ ]:
print(torch.__version__)

2.4.1+cu121


In [ ]:
image_size = (64, 64)  # Resize images to 64x64
n_classes = 3  # 3 classes: COVID, Normal, Viral
n_qubits = 4   # Number of qubits for quantum classifier
n_layers = 4   # Number of layers in the quantum circuit
epochs = 10    # Training epochs
batch_size = 16 # Batch size for training
images_per_class = 160

In [ ]:
dataset_dir = "/content/drive/MyDrive/Finalproject/dataset"
classes = ['COVID', 'Normal', 'Viral']

In [ ]:
# Initialize lists to hold the data and labels
data = []
labels = []

# Loop through each class subfolder
for class_name in classes:
    class_path = dataset_dir + '/' + class_name

    # Ensure the class folder exists
    if not os.path.isdir(class_path):
        print(f"Directory {class_path} does not exist.")
        continue

    # List all image files in the class subfolder
    image_files = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]

    # Shuffle the files to ensure randomness
    random.shuffle(image_files)

    # Select 200 images only (or less if fewer images are available)
    selected_images = image_files[:images_per_class]

    # Process each image
    for image_file in selected_images:
        # Load the image
        image_path = os.path.join(class_path, image_file)
        image = cv2.imread(image_path)

        # Resize image (optional, based on your model requirements)
        if image is not None:
            image = cv2.resize(image, image_size)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            data.append(image)
            labels.append(class_name)
        else:
            print(f"Error loading image: {image_path}")

In [ ]:
data = np.array(data)
labels = np.array(labels)

In [ ]:
data = data/255.0

In [ ]:
print(data.shape)
print(labels.shape)

(480, 64, 64)
(480,)


In [ ]:
print(type(data))
print(type(labels))

<class 'pennylane.numpy.tensor.tensor'>
<class 'pennylane.numpy.tensor.tensor'>


In [ ]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [ ]:
data_torch = torch.tensor(data, dtype=torch.float32)
labels_torch = torch.tensor(labels, dtype=torch.long)

In [ ]:
data_torch = data_torch.unsqueeze(1)

In [ ]:
data_torch.shape

torch.Size([480, 1, 64, 64])

In [ ]:
# labels_one_hot = torch.nn.functional.one_hot(labels_torch, num_classes=n_classes).float()

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_torch = data_torch.to(device)
labels_torch = labels_torch.to(device)

In [19]:
print(data_torch.shape)
print(labels_torch.shape)

torch.Size([480, 1, 64, 64])
torch.Size([480])


In [20]:
from torch.utils.data import DataLoader, TensorDataset

# Create a dataset and dataloader
dataset = TensorDataset(data_torch, labels_torch)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [21]:
for images, labels in dataloader:
    # Move images and labels to the device if not already
    images = images.to(device)
    labels = labels.to(device)

In [22]:
n_qubits = 4
dev = qml.device("default.mixed", wires=n_qubits)

In [23]:
@qml.qnode(dev)
def qnode(inputs, weights):
    for depth in range(4):
        for i in range(n_qubits):
            qml.Hadamard(wires=i)
            qml.RY(inputs[i], wires=i)
            qml.RZ(inputs[i], wires=i)
            qml.RX(inputs[i], wires=i)
            qml.RY(inputs[i], wires=i)

        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[2, 3])
        qml.CNOT(wires=[1, 2])

        for i in range(n_qubits):
            qml.RY(weights[0][i], wires=i)
            qml.RZ(weights[1][i], wires=i)
            qml.RX(weights[2][i], wires=i)
            qml.RY(weights[3][i], wires=i)


    return [qml.expval(qml.PauliZ(wires=i)) for i in range(3)]


In [24]:
n_layers = 4
weight_shapes = {"weights": (n_layers, n_qubits)}

In [25]:
qlayer = qml.qnn.TorchLayer(qnode, weight_shapes).to(device)

In [26]:
class HybridModel(nn.Module):
    def __init__(self):
        super(HybridModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=2, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=2, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=2, padding=1)
        self.bn3 = nn.BatchNorm2d(32)
        self.fc1 = nn.Linear(32 * 8 * 8, 1024)  # Adjust based on input size
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 16)
        self.qlayer = qlayer
        self.output = nn.Linear(3, n_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.5)
        x = F.relu(self.fc3(x))
        x = self.qlayer(x)
        x = torch.reshape(x, (x.size(0), -1))
        x = self.output(x)
        return x


In [27]:
model = HybridModel().to(device)

In [28]:
print(device)

cuda


In [29]:
print(model)

HybridModel(
  (conv1): Conv2d(1, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=2048, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=16, bias=True)
  (qlayer): <Quantum Torch Layer: func=qnode>
  (output): Linear(in_features=3, out_features=3, bias=True)
)


In [ ]:
# model.state_dict()

In [30]:
from torch.optim import Adam

In [31]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.0001)  # Adjusted learning rate

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = correct / total * 100
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

print("Training complete!")


Epoch [1/10], Loss: 1.1178, Accuracy: 33.54%
Epoch [2/10], Loss: 1.1142, Accuracy: 34.38%
Epoch [3/10], Loss: 1.1121, Accuracy: 32.71%
Epoch [4/10], Loss: 1.1158, Accuracy: 33.96%
Epoch [5/10], Loss: 1.1140, Accuracy: 32.92%
Epoch [6/10], Loss: 1.1134, Accuracy: 33.33%
Epoch [7/10], Loss: 1.1129, Accuracy: 33.96%
Epoch [8/10], Loss: 1.1072, Accuracy: 32.50%
Epoch [9/10], Loss: 1.1062, Accuracy: 34.38%
Epoch [10/10], Loss: 1.1182, Accuracy: 30.63%
Training complete!


In [ ]:
# model.state_dict()

OrderedDict([('conv1.weight',
              tensor([[[[-0.2480,  0.3517],
                        [-0.2035, -0.2497]]],
              
              
                      [[[-0.4381, -0.3219],
                        [-0.3220,  0.2392]]],
              
              
                      [[[ 0.1475, -0.3571],
                        [ 0.2318, -0.0213]]],
              
              
                      [[[-0.1538, -0.3320],
                        [-0.0327, -0.1388]]],
              
              
                      [[[-0.4098, -0.1546],
                        [-0.4349, -0.3135]]],
              
              
                      [[[ 0.1036,  0.0581],
                        [-0.0572,  0.2856]]],
              
              
                      [[[-0.4764,  0.0927],
                        [-0.3887,  0.2466]]],
              
              
                      [[[-0.3917,  0.2895],
                        [ 0.1155, -0.2325]]],
              
              
          

In [32]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=2, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=2, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=2, padding=1)
        self.bn3 = nn.BatchNorm2d(32)
        self.fc1 = nn.Linear(32 * 8 * 8, 1024)  # Adjust based on input size
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 16)
        self.output = nn.Linear(16, n_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.5)
        x = F.relu(self.fc3(x))
        x = self.output(x)
        return x


In [33]:
model2 = CNNModel().to(device)

In [35]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model2.parameters(), lr=0.0001)  # Adjusted learning rate

# Training loop
epochs = 20
for epoch in range(epochs):
    model2.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model2(inputs)
        loss = criterion(outputs, targets)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = correct / total * 100
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

print("Training complete!")


Epoch [1/20], Loss: 0.2429, Accuracy: 90.62%
Epoch [2/20], Loss: 0.2071, Accuracy: 91.46%
Epoch [3/20], Loss: 0.1518, Accuracy: 95.83%
Epoch [4/20], Loss: 0.1436, Accuracy: 95.21%
Epoch [5/20], Loss: 0.1377, Accuracy: 96.04%
Epoch [6/20], Loss: 0.1376, Accuracy: 94.79%
Epoch [7/20], Loss: 0.1266, Accuracy: 94.17%
Epoch [8/20], Loss: 0.0916, Accuracy: 98.12%
Epoch [9/20], Loss: 0.0950, Accuracy: 97.08%
Epoch [10/20], Loss: 0.0891, Accuracy: 97.29%
Epoch [11/20], Loss: 0.0668, Accuracy: 98.12%
Epoch [12/20], Loss: 0.0558, Accuracy: 98.12%
Epoch [13/20], Loss: 0.0563, Accuracy: 98.12%
Epoch [14/20], Loss: 0.0361, Accuracy: 99.17%
Epoch [15/20], Loss: 0.0543, Accuracy: 97.71%
Epoch [16/20], Loss: 0.0440, Accuracy: 98.54%
Epoch [17/20], Loss: 0.0545, Accuracy: 98.12%
Epoch [18/20], Loss: 0.0450, Accuracy: 98.96%
Epoch [19/20], Loss: 0.0205, Accuracy: 100.00%
Epoch [20/20], Loss: 0.0282, Accuracy: 99.38%
Training complete!
